In [ ]:
"""
This notebook walks though how to use BuildingMotif Templates.
"""
from rdflib import RDF, URIRef

from buildingmotif.building_motif import BuildingMotif
from buildingmotif.dataclasses.template import Template
from buildingmotif.dataclasses.template_library import TemplateLibrary

In [ ]:
"""
As always with BuildingMotif, we start by initing BuildingMotif.
"""
building_motif = BuildingMotif(f"sqlite://")
building_motif.session


In [ ]:
"""
Templates must always be conneted to a template library.
"""
tl = TemplateLibrary.create("my_tl")
tl

In [ ]:
"""
Create a template through the template library. The body is currently empty.

"""
ahu_template = tl.create_template(
    name="ahu",
    head=["name", "zone"],
)

print(ahu_template)
print(ahu_template.head)
print(ahu_template.body.serialize(format="ttl"))

In [ ]:
"""
Now we can populate the body.
Note that the nodes "name" and "zone" are variables, or undefinied.
They are the template's head, as defined above. This means their values
will be pass to the template when evaluated.
"""
def as_brick(x):
    return URIRef(f"https://brickschema.org/schema/Brick#{x}")

def as_var(x):
    return URIRef(f"https://buildingmotif.org/var#{x}")

ahu_template.body.add((as_var("name"), RDF.type, as_brick("AHU")))
ahu_template.body.add((as_var("name"), as_brick("feeds"), as_var("zone")))


print(ahu_template.body.serialize(format="ttl"))

In [ ]:
"""
Note that the head is immutable.
"""
try:
    ahu_template.head = ["nope"]
except AttributeError as e:
    print(e)

try:
    ahu_template.head.append("nope")
except AttributeError as e:
    print(e)

In [ ]:
"""
In addition to heads, templates may also have dependancies on other templates.
Say we added to ahu_template the following tuple, where "sf" refers to the
following supply_fan template.

We must then create a dependancy between the two, and define how the dependancy
head should be evaluated.
"""
# this new triples requires dependancy "sf"
ahu_template.body.add((as_var("name"), as_brick("hasPart"), as_var("sf")))

# here we define the template we will depend on
sf_template = tl.create_template(
    name="supply_fan",
    head=["name"],
)
sf_template.body.add((as_var("name"), RDF.type, as_brick("Supply_Fan")))

# and finally define the dependancy.
ahu_template.add_dependency(sf_template, {"name": "sf"})

print(ahu_template.get_dependencies())

In [ ]:
""" 
Note that both templates must be part of the same library, 
all heads must be defined to create a dependancy,
and that there can only be one dependency per directional dependant/dependee pair.
"""
building_motif.session.commit()

# not in same library
try:
    other_tl = tl = TemplateLibrary.create("other_tl")
    other_sf_template = tl.create_template(
        name="supply_fan",
        head=["name"],
    )
    ahu_template.add_dependency(other_sf_template, {"name": "sf"})
except Exception as e:
    print(e)
    building_motif.session.rollback()
    
# bad head
try:
    ahu_template.add_dependency(sf_template, {"not a head": "sf"})
except ValueError as e:
    print(e)

# already exist
try:
    ahu_template.add_dependency(sf_template, {"name": "sf"})
except Exception as e:
    print(e)
    building_motif.session.rollback()


# Loading Templates from an Ontology

There is initial support for loading templates from the Class and NodeShape definitions contained within an ontology:

In [ ]:
lib = building_motif.load_library("../tests/fixtures/Brick1.3rc1-equip-only.ttl")
print(lib)

In [ ]:
"""TO COME: template evaluation"""